In [ ]:
!pip install -q openai # Install OpenAI 

In [ ]:
%reset -f # Reset environment

In [3]:
import os
from openai import OpenAI           # import OpenAI

key = os.environ.get('API_KEY')          # Get key from Windows Environment

client = OpenAI(
    # This is the default and can be omitted
    api_key = key
)

In [5]:
# Text completion
def my_ai_assistant(prompt):
    response = client.chat.completions.create(
       messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4-turbo",
)
    return response.choices[0].message

# Example usage
prompt = "Translate the following English text to French: 'Hello, how are you? I am testing the Code'"
result = my_ai_assistant(prompt)
print("My AI's response:", result)

My AI's response: ChatCompletionMessage(content='Bonjour, comment vas-tu ? Je teste le Code.', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [ ]:
# Function to upload a dataset
from pathlib import Path

file = client.files.create(
    file =  Path('sample_data/cacao.csv'),
    purpose='assistants'
)
file.id

In [ ]:
# Create an assistant using the file ID
assistant = client.beta.assistants.create(
  instructions="OLSAI",
  model="gpt-4-turbo",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }
)

In [ ]:
# Create an empty thread
thread = client.beta.threads.create()
  #messages=[
   # {
    #  "role": "user",
     # "content": "List summary statistics",
      #"attachments": [
       # {
        #  "file_id": file.id,
         # "tools": [{"type": "code_interpreter"}]
        #}
      #]
    #}

  #]
#)


In [ ]:
thread_message = client.beta.threads.messages.create(
  thread.id,
  role="user",
  content="Make summary statistics",
  attachments=[
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }]
)
print(thread_message)

In [ ]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Student."
)

In [ ]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

In [ ]:
messages.data[0].content

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

for step in run_steps.data:

  # If step involves code execution, print the code
  if step.type == 'code_interpreter':
      print(f"Python Code Executed: {step.step_details['code_interpreter']['input']}")
